建立和网络设备的安全连接

In [ ]:
import paramiko

client = paramiko.SSHClient() #创建SSH对象

client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) #设置自动添加主机密钥策略
client.connect(
    hostname='192.168.56.102',
    port=22,
    username='admin',
    password='Admin@h3c.com',
    look_for_keys=False,
    allow_agent=False,
)

client.close


<bound method SSHClient.close of <paramiko.client.SSHClient object at 0x00000176FE04D250>>

异步执行命令并处理响应

In [ ]:
import re
import paramiko
import time

try:
    client = paramiko.SSHClient() #创建SSH对象
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) #设置自动添加主机密钥策略
    client.connect(
        hostname='192.168.56.100',
        port=22,
        username='admin',
        password='Admin@h3c.com',
        look_for_keys=False,
        allow_agent=False,
        timeout=3,
    )

    ssh_client = client.invoke_shell() #创建一个交互式shell对象
    ssh_client.send('screen-length disable\n') #发送命令
    time.sleep(1) #等待命令执行完成
    ssh_client.recv(65000) #接收命令执行结果
    ssh_client.send('dis vlan\n') #发送命令
    time.sleep(2) 
    buff = ''
    while True:
        Chunk = ssh_client.recv(65000).decode(errors='ignore') #忽略编码错误，防止特殊字符抛异常
        buff += Chunk
        if re.search(r'<[\w-]+>|\[[\w-]+\]',buff.splitlines()[-1]):#如果buff的最后一行以<或[开头，则停止接收
            break
        print('\n'.join(buff.splitlines()[1:-1])) 
    print(buff)
    ssh_client.close()
    client.close
except paramiko.AuthenticationException as e:
    print(f"认证失败: {e}")
except paramiko.SSHException as e:
    print(f"SSH连接失败: {e}")
except Exception as e:
    print(f"发生错误: {e}")



发生错误: timed out


连接到多个设备

In [ ]:
import re
import paramiko
import time

try:
    client = paramiko.SSHClient() #创建SSH对象
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    for device in range(102,104 + 1):
        host = "192.168.56." + str(device)
        print(f"连接到{host}...")
        client.connect(
            hostname=host,
            port=22,
            username='admin',
            password='Admin@h3c.com',
            look_for_keys=False,
            allow_agent=False,
            timeout=3,
        )
        ssh_client = client.invoke_shell() #创建一个交互式shell对象
        ssh_client.send('screen-length disable\n') #发送命令
        time.sleep(1) #等待命令执行完成
        ssh_client.recv(65000) #接收命令执行结果
        ssh_client.send('dis vlan\n') #发送命令
        time.sleep(2)
        buff = ''
        while True:
            Chunk = ssh_client.recv(65000).decode(errors='ignore') #忽略编码错误，防止特殊字符抛异常
            buff += Chunk
            if re.search(r'<[\w-]+>|\[[\w-]+\]',buff.splitlines()[-1]):#如果buff的最后一行以<或[开头，则停止接收
                break
            print('\n'.join(buff.splitlines()[1:-1]))
        print(buff)
        ssh_client.close()
    client.close
except paramiko.AuthenticationException as e:
    print(f"认证失败: {e}")
except paramiko.SSHException as e:
    print(f"SSH连接失败: {e}")
except Exception as e:
    print(f"发生错误: {e}")
        
        

文件上传

In [19]:
import paramiko

def sftp_upload_file(host,user,passsword,server_path,local_path,timeout=10):
    try:
        transport = paramiko.Transport((host,22))
        transport.connect(username=user,password=password)
        sftp = paramiko.SFTPClient.from_transport(transport)
        sftp.put(local_path,server_path)
        sftp.close()
        return True
    except Exception as e:
        print(f"文件上传失败: {e}")
        return False
    
if __name__ == "__main__":
    host = '192.168.56.102'
    user = 'admin'
    password = 'Admin@h3c.com'
    server_path = '/id_switch.pub'
    local_path = r'C:\Users\shenchao\.ssh\id_switch.pub'
    res = sftp_upload_file(host,user,password,server_path,local_path)
    if not res:
        print("文件上传: %s 失败" % local_path)
    else:
        print("文件上传: %s 成功" % local_path)
        
        


文件上传: C:\Users\shenchao\.ssh\id_switch.pub 成功


下载文件

In [ ]:
import paramiko

def sftp_download_file(host, user, password, server_path, local_path, timeout=10):
    """
    从 H3C 设备通过 SFTP 下载文件
    :param host: H3C 设备 IP 地址
    :param user: 用户名
    :param password: 密码
    :param server_path: 设备上的文件路径（需使用 Linux 风格路径）
    :param local_path: 本地保存路径
    :param timeout: 连接超时时间（秒）
    :return: 成功返回 True，失败返回 False
    """
    try:
        # 创建传输对象并连接
        transport = paramiko.Transport((host, 22))
        transport.connect(username=user, password=password)

        # 创建 SFTP 客户端
        sftp = paramiko.SFTPClient.from_transport(transport)

        # 执行下载操作（注意：是 get，不是 put）
        sftp.get(server_path, local_path)
        
        # 关闭连接
        sftp.close()
        transport.close()

        return True

    except Exception as e:
        print(f"文件下载失败：{e}")
        return False


if __name__ == "__main__":
    host = '192.168.56.102'
    user = 'admin'
    password = 'Admin@h3c.com'
    
    # 修正路径：使用 Linux 风格的绝对路径，去掉 'flash:' 前缀
    server_path = '/logfile/logfile.log'
    
    # 本地保存路径
    local_path = r'C:\Users\shenchao\.ssh\2026_01_05_logfile.log'

    # 执行下载
    res = sftp_download_file(host, user, password, server_path, local_path)
    
    if not res:
        print(f"文件下载：{local_path} 失败")
    else:
        print(f"文件下载：{local_path} 成功")

文件下载：C:\Users\shenchao\.ssh\2026_01_05_logfile.log 成功


Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
